In [5]:
import gensim
import pandas as pd
import jieba
import re
import stanfordcorenlp
import math
import numpy as np
import random

## Data preprocessing

In [6]:
data_source = "D:/Repos/MyPython/Nlp/sqlResult_1558435.csv"

In [7]:
data = pd.read_csv(data_source,encoding='gb18030')

In [8]:
data = data.fillna('')#给为null的数据赋''

In [9]:
def cut(string): return ' '.join(jieba.cut(string))

In [9]:
cut("你今天运动了吗")

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\jeffb\AppData\Local\Temp\jieba.cache
Loading model cost 1.162 seconds.
Prefix dict has been built successfully.


'你 今天 运动 了 吗'

In [10]:
content = data['content'].tolist()

In [11]:
def token(string):
    return re.findall(r'[\d|\w]+',string)

In [12]:
token("这是一个测试\n\n\n")

['这是一个测试']

In [12]:
news_content = [token(n) for n in content]

In [13]:
news_content = [''.join(n) for n in news_content]

In [14]:
news_content = [cut(n) for n in news_content]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jeffb\AppData\Local\Temp\jieba.cache
Loading model cost 0.952 seconds.
Prefix dict has been built successfully.


In [15]:
news_content[:1]

['此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息']

## TF-IDF

In [19]:
def document_frequency(word):
    return sum(1 for n in news_content if word in n)

In [20]:
def idf(word):
    return math.log10(len(news_content)/document_frequency(word))

In [21]:
idf("的")

0.1051466115514474

In [19]:
idf("小米")

2.9523613237924735

In [22]:
def tf(word,document):
    words = document.split()
    return sum(1 for w in words if w==word)

In [23]:
tf("的",news_content[0])

2

In [24]:
def tf_idf(word,document):
    return tf(word,document)*idf(word)

In [25]:
tf_idf("的",news_content[0])

0.2102932231028948

In [26]:
def get_keywords_of_a_document(document):
    words = set(document.split())
    tfidf = [
        (w,tf_idf(w,document)) for w in words
    ]
    tfidf = sorted(tfidf,key=lambda x: x[1],reverse=True)
    return tfidf

In [27]:
get_keywords_of_a_document(news_content[1])

[('骁龙', 14.28860032832347),
 ('Windows10', 12.14781401040159),
 ('桌面', 8.674990450316585),
 ('高通', 6.392972936239964),
 ('835', 6.363018624300659),
 ('微软', 5.379820468124088),
 ('CPUGPU', 4.9523613237924735),
 ('二合一', 4.9523613237924735),
 ('biglittle', 4.9523613237924735),
 ('835Win10', 4.9523613237924735),
 ('性能', 4.756660112129509),
 ('源码', 4.651331328128492),
 ('Wintel', 4.651331328128492),
 ('屏蔽掉', 4.651331328128492),
 ('SoC', 4.475240069072811),
 ('基带', 4.475240069072811),
 ('Exynos', 4.350301332464511),
 ('Q4', 4.17421007340883),
 ('联发科', 4.107263283778217),
 ('ARM', 4.04927133680053),
 ('PCB', 3.9981188143531488),
 ('华硕', 3.9523613237924735),
 ('平台', 3.8983153086735953),
 ('功耗', 3.6301420290585544),
 ('惠普', 3.6099386429702673),
 ('蓝牙', 3.5721500820808676),
 ('麒麟', 3.4472113454725677),
 ('WiFi', 3.3503013324645115),
 ('处理器', 3.280263465856756),
 ('方案', 3.105375204622871),
 ('小米', 2.9523613237924735),
 ('联想', 2.91493482585185),
 ('按计划', 2.694682748923289),
 ('架构', 2.6736077228396

In [28]:
news_content[1]

'骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台'

### TFIDF Vectorizer

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorizer = TfidfVectorizer(max_features=10000)

In [31]:
sample_num = 50000
sub_samples = news_content[:sample_num]

In [32]:
X = vectorizer.fit_transform(sub_samples)

In [33]:
len(vectorizer.vocabulary_)

10000

In [34]:
np.where(X[1].toarray())  # find positions whose values is not zero 

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([ 100,  281,  360,  372,  472,  578, 1008, 1162, 1244, 1269, 1285,
        1380, 1462, 1886, 2435, 2754, 2760, 2989, 3067, 3353, 3800, 3990,
        4173, 4272, 4429, 4497, 4521, 4529, 4595, 4658, 4961, 5037, 5104,
        5136, 5550, 5717, 5771, 5899, 6035, 6081, 6710, 6971, 7072, 7217,
        7506, 7986, 8025, 8026, 8137, 8207, 8479, 8558, 8799, 9127, 9196,
        9252, 9637, 9744, 9824], dtype=int64))

In [35]:
X[1].toarray()[0][12]

0.0

In [36]:
document_1,document_2 = random.randint(0,1000),random.randint(0,1000)

In [37]:
document_1

655

In [38]:
document_2

590

In [39]:
news_content[document_1]

'新华社 博鳌 6 月 23 日电 记者 赵叶 苹以 新形势下 加强 和 改进 党 对 新闻 舆论 工作 领导 的 经验 做法 为 主题 的 第六次 中老 两党 理论 研讨会 23 日 在 海南 博鳌 闭幕 中共中央 网信 办 副 主任 任 贤良 和 老挝 国家 政治 行政 学院 院长 通沙立 分别 致 闭幕 辞中 老 双方 均 表示 本次 研讨会 对于 加强 两党 在 新闻 舆论 工作 领域 的 经验交流 互鉴 进一步 把握 新形势下 新闻 舆论 工作 的 特点 和 规律 提升 党 对 新闻 舆论 工作 的 领导 能力 和 水平 对于 各自 国家 的 社会主义 现代化 建设 事业 的 发展 具有 重要 现实意义 在 为期 两天 的 研讨会 上 中 老 双方 代表 畅所欲言 就 新形势下 坚持 党管 媒体 坚持 妥善 引导 舆论 坚持 媒体 融合 发展 深化 双方 交流 等 重要 问题 进行 了 深入 讨论 达成 广泛 共识'

In [40]:
vector_of_document_1 = X[document_1].toarray()[0]

In [41]:
news_content[document_2]

'本文 来源 央视 新闻 鹘 鹰 战机 的 模型 亮相 巴黎 航展 第 52 届 巴黎 航展 于 6 月 19 日至 25 日 举行 我国 最新型 多用途 隐身 战机 鹘 鹰 也 就是 歼 31 战机 首次 亮相 巴黎 航展 引发 各界 关注 被 称为 鹘 鹰 20 版 的 中国 歼 31 战机 歼 31 战斗机 代号 鹘 鹰 研制 单位 中航 工业 沈阳 飞机 工业 集团公司 首飞 时间 2012 年 10 月 布局 单座 双发 外倾 双垂尾 全动 平尾 定位 主导 未来 战场 的 最新型 多用途 隐身 战斗机 特性 高 生存力 和 多任务 能力 高 生存力 低 雷达 可 探测 性低 红外 辐射 特性 优异 的 电子对抗 性能 低 易损性 设计 多任务 能力 强大 的 目标 探测 和 外部 信息 综合 能力 优异 的 态势 感知 和 信息 共享 能力 超 视距 多 目标 攻击 和 大离 轴角 全 向 攻击 格斗 空战 能力 对 地 和 海面 目标 精确 打击 能力 适应 复杂 气象条件 和 广 地域 使用 能力 优异 的 综合 保障 能力 中国 航空工业 集团公司 副总经理 李玉海 表示 鹘 鹰 战机 具备 高 隐身 高 敏捷性 超音速 等 优势 其 技术 成熟度 已经 相当 高 李玉海 介绍 鹘 鹰 作为 一款 我国 具有 完全 自主 知识产权 的 第五代 战机 在 性能 上 与 同 等级 别的 战机 没有 差距 与 第四代 战机 相比 更是 具有 压倒性 优势 另外 这个 量级 的 飞机 中 无论是 单机 成本 还是 全 寿命 机 的 成本 鹘 鹰 都 处于 领先水平 也就是说 性能 不 差且 性价比 高 此前 鹘 鹰 战机 还 曾 参加 过 2015 年 的 迪拜 航展 2016 年 被 称为 鹘 鹰 20 的 改进型 歼 31 试飞 成功 改进 版本 不仅 改进 了 空气 动力 设计 还 加大 了 武器 舱 军事 专家 尹卓 曾 分析 歼 31 离 定型 还 需要 一段时间 未来 不 排除 成为 航空母舰 舰载机 此次 巴黎 航展 中国 围绕 军机 民机 和 机载设备 三条 主线 共 组织 9 大类 15 项 重点 产品 和 系统 参展 除鹘鹰 战机 外 中国 航空工业 的 翼 龙 系列 中 高阶 军用 无人机 也 将 在 本届 巴黎 航空展 亮相 包括 

In [42]:
vector_of_document_2 = X[document_2].toarray()[0]

In [43]:
from scipy.spatial.distance import cosine

In [44]:
def distance(v1,v2): return cosine(v1,v2)

In [45]:
distance(vector_of_document_1,vector_of_document_2)

0.9557465539131513

## TextRank   homework

## LDA

In [3]:
from gensim import corpora, models
import jieba.posseg as jp,jieba#词性标注

In [17]:
#	n	名词
#	nr	人名
#   ns	地名
#   nt	机构团体
#   eng 英语?
#   v	动词
#   d	副词
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')

In [50]:
data = pd.read_csv(data_source,encoding='gb18030')

In [18]:
news = data["content"][:100]

In [19]:
stop_words = []
with open('./百度停用词表.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [21]:
words_ls = []
for text in news:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]
    words_ls.append(words)

In [22]:
len(words_ls)

100

In [23]:
dictionary = corpora.Dictionary(words_ls)

In [24]:
corpus = [dictionary.doc2bow(words) for words in words_ls]

In [27]:
corpus[0]

[(0, 3),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 1)]

In [28]:
lda = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=5)

In [29]:
for topic in lda.print_topics(num_words=10):
    print(topic)

(0, '0.007*"市场" + 0.006*"不" + 0.005*"都" + 0.005*"还" + 0.005*"会" + 0.005*"格力" + 0.005*"手机" + 0.004*"挡泥板" + 0.004*"人" + 0.004*"中国"')
(1, '0.008*"企业" + 0.006*"市场" + 0.006*"跨境" + 0.005*"乐视" + 0.005*"都" + 0.004*"不" + 0.004*"还" + 0.004*"人" + 0.003*"中国" + 0.003*"会"')
(2, '0.006*"中国" + 0.005*"都" + 0.004*"还" + 0.004*"会" + 0.004*"不" + 0.004*"人" + 0.004*"市场" + 0.003*"内容" + 0.003*"手机" + 0.003*"资金"')
(3, '0.006*"会" + 0.005*"都" + 0.005*"冰架" + 0.005*"乐视" + 0.004*"人" + 0.004*"不" + 0.004*"市场" + 0.004*"中国" + 0.003*"记者" + 0.003*"内容"')
(4, '0.005*"会" + 0.005*"市场" + 0.004*"人" + 0.004*"都" + 0.004*"不" + 0.003*"还" + 0.003*"记者" + 0.003*"女子" + 0.003*"相关" + 0.003*"摄像头"')


In [30]:
text = data["content"][102]

In [31]:
words = [[w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]]

In [32]:
text_corpus = [dictionary.doc2bow(word) for word in words]

In [33]:
list(lda[text_corpus])

[[(0, 0.25606662), (1, 0.2427181), (2, 0.43912604), (3, 0.059972305)]]

In [34]:
text

'新疆日报讯（通讯员吕丽娜报道）6月13日，笔者在拜城县残联培训基地见到了正在参加雕刻技术培训的托乎提·斯坎达尔，今年31岁的他是拜城县托克逊乡亚吐尔村的村民。通过学习，他已经可以利用残疾的双手雕刻出漂亮的葫芦花纹，学到技术后的他还加入了农民合作社，现在每个月有2000多元的稳定收入，最高时月收入达5000多元。\r\n据悉，今年以来，拜城县为进一步做好残疾人就业援助工作，采取“兴趣+技能”“培训+技能”等方式，在该县残疾人培训基地分别举办厨师班、刺绣班、雕刻班、地毯编织班等技能培训班，给全县的残疾人提供学习技能的机会，学习期间为他们免费提供食宿。\r\n现在每个班都会吸引一些残疾人前来学习技术，技能培训班不仅能帮助残疾人掌握一门技能，还能拓宽残疾人就业渠道。家住拜城县大桥乡农民阿不都热依木·亚森说：“我现在在合作社上班，有了稳定收入，家里的变化也非常大。谢谢党和政府的好政策！”\r\n截至目前，全县已有50多名残疾人成功实现就业。\r\n免责声明：本文仅代表作者个人观点，与环球网无关。其原创性以及文中陈述文字和内容未经本站证实，对本文以及其中全部或者部分内容、文字的真实性、完整性、及时性本站不作任何保证或承诺，请读者仅作参考，并请自行核实相关内容。\r\n'